# Analysis on cleaned dataset from Data.ipynb

In [ ]:
#only perform once in the beginning
#%pip install linearmodels

In [3]:
import pandas as pd
import numpy as np

# Fixed effects regression:
from linearmodels import PanelOLS
import statsmodels.api as sm


In [19]:
# import the csv data from Data.ipynb
data = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/data_clean.csv')
houshold = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/houshold_clean.csv', index_col=['SSUID', 'month_total'])

# Analysis base modification
- with & without groupings to make sure the definition is correct
- with my dataset and with POWELL dataset
- with time and entity effect


In [20]:
houshold.head()

TFEARN  erbamth  TFTOTINC  ERBATAMT  ems  EHHNUMPP  \
SSUID       month_total                                                       
19128000276 6.0               0      NaN      1099       0.0    1         3   
            7.0               0      NaN      1099       0.0    1         3   
            8.0               0      NaN      1099       0.0    1         3   
            9.0               0      NaN      1099       0.0    1         3   
            10.0              0      NaN       381       0.0    1         3   

                         erebate-6  erebate-5  erebate-4  erebate-3  ...  \
SSUID       month_total                                              ...   
19128000276 6.0                0.0        0.0        0.0        0.0  ...   
            7.0                0.0        0.0        0.0        0.0  ...   
            8.0                0.0        0.0        0.0        0.0  ...   
            9.0                0.0        0.0        0.0        0.0  ...   
            10.0               0.0        0.0        0.0        0.0  ...   

                         erebate2  erebate3  erebate4  erebate5  erebate6  \
SSUID       month_total                                                     
19128000276 6.0               0.0       0.0       0.0       0.0       0.0   
            7.0               0.0       0.0       0.0       0.0       0.0   
            8.0               0.0       0.0       0.0       0.0       0.0   
            9.0               0.0       0.0       0.0       0.0       0.0   
            10.0              0.0       0.0       0.0       0.0       0.0   

                         erebate7  erebate8  erebate9  erebate10  erebate11  
SSUID       month_total                                                      
19128000276 6.0               0.0       0.0       0.0        0.0        0.0  
            7.0               0.0       0.0       0.0        0.0        0.0  
            8.0               0.0       0.0       0.0        0.0        0.0  
            9.0               0.0       0.0       0.0        0.0        0.0  
            10.0              0.0       0.0       0.0        0.0        0.0  

[5 rows x 24 columns]

In [18]:
# put this to the Data.ipynb # -> or should I drop all observations with earnings below 0?
print(houshold['TFEARN'].describe())
houshold.loc[houshold.TFEARN < 0, 'TFEARN'] = 0
print(houshold['TFEARN'].describe())

count    209904.000000
mean       5153.436137
std        5758.693401
min      -46667.000000
25%        1577.000000
50%        3810.000000
75%        7000.000000
max       98083.000000
Name: TFEARN, dtype: float64
count    209904.000000
mean       5166.651622
std        5734.088956
min           0.000000
25%        1577.000000
50%        3810.000000
75%        7000.000000
max       98083.000000
Name: TFEARN, dtype: float64


In [19]:
# with dummy * amount:
exog_vars = houshold[['erebate_-2.0', 'erebate_-1.0', 'erebate_0.0', 'erebate_1.0','erebate_2.0', 'erebate_3.0', 'erebate_4.0', 'erebate_5.0', 'erebate_6.0', 
                      'erebate_7.0', 'erebate_8.0', 'erebate_9.0', 'erebate_10.0', 'erebate_11.0']].multiply(houshold['ERBATAMT'], axis = 'index')
#exog_vars['month_total'] = pd.Categorical(houshold['month_total'])
exog = sm.add_constant(exog_vars)

model = PanelOLS(houshold['TFEARN'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,TFEARN,R-squared:,8.375e-05
Estimator:,PanelOLS,R-squared (Between):,-0.0007
No. Observations:,209904,R-squared (Within):,0.0002
Date:,"Tue, May 31 2022",R-squared (Overall):,-0.0006
Time:,16:56:06,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.0987
Entities:,26238,P-value,0.3526
Avg Obs:,8.0000,Distribution:,"F(14,183642)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.0987


In [20]:
# die obere regression nochmal machen aber rebate rebate0+rebate1 und rebate2+rebate3 gruppieren: 
# addieren der dummies, die rows bleiben aber so bestehen und werden nicht aggregiert

powell_hh = houshold[['ERBATAMT', 'TFEARN']]
powell_hh['reb'] = houshold['erebate_-1.0'] + houshold['erebate_0.0']
powell_hh['reb_lag'] = houshold['erebate_1.0'] + houshold['erebate_1.0']
powell_hh['reb_lag1'] = houshold['erebate_4.0'] + houshold['erebate_3.0']
powell_hh['reb_lag2'] = houshold['erebate_5.0'] + houshold['erebate_6.0'] + houshold['erebate_7.0'] + houshold['erebate_8.0']

exog_vars = powell_hh[['reb', 'reb_lag']].multiply(powell_hh['ERBATAMT'], axis = 'index') #, 'reb_lag1', 'reb_lag2'

exog = sm.add_constant(exog_vars)

model = PanelOLS(powell_hh['TFEARN'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()


# include interaction terms? 
# what is the 2SLS approach in Powell? is the first stage Rebate = gamma (interaction terms of HH characteristica)?
# read the stata code again (especially concerning the interacgtion term!)
# set tfearn = 0 if tfearn < 0.

# don't use houshold dataframe that is grouped but use the regular one with pp as index and cluster by hh in the regressions.

<ipython-input-20-f2f994139ad6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb'] = houshold['erebate_-1.0'] + houshold['erebate_0.0']
<ipython-input-20-f2f994139ad6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb_lag'] = houshold['erebate_1.0'] + houshold['erebate_1.0']
<ipython-input-20-f2f994139ad6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Dep. Variable:,TFEARN,R-squared:,1.347e-05
Estimator:,PanelOLS,R-squared (Between):,6.237e-05
No. Observations:,209904,R-squared (Within):,-3.647e-05
Date:,"Tue, May 31 2022",R-squared (Overall):,5.056e-05
Time:,16:56:28,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.2373
Entities:,26238,P-value,0.2902
Avg Obs:,8.0000,Distribution:,"F(2,183654)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.2373


In [ ]:
# with dummy only:
exog_vars = houshold[['erebate_-2.0', 'erebate_-1.0', 'erebate_0.0', 'erebate_1.0','erebate_2.0', 'erebate_3.0', 'erebate_4.0', 'erebate_5.0', 'erebate_6.0', 
                      'erebate_7.0', 'erebate_8.0', 'erebate_9.0', 'erebate_10.0', 'erebate_11.0']]
exog_vars['month_total'] = pd.Categorical(houshold['month_total'])
exog = sm.add_constant(exog_vars)

model = PanelOLS(houshold['TFEARN'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()


# there is a cluster option as in the stata code

In [28]:
# next step: estimation with grouping:
totinc_dict = {
    1: range(0,2499),
    2: range(2500,5999),
    3: range(5000,12499),
    4: range(12500,99999)
}

reb_dummies_lst = ['erebate-1', 'erebate0', 'erebate1','erebate2', 'erebate3', 'erebate4']
totinc_dict.keys()

dict_keys([1, 2, 3, 4])

In [38]:
# alt:
def get_grouping(df_column, group_dict, reb_dummies_df):
    # get flag indicating the group with the dictionary-mapping:
    new_col = df_column.apply(lambda x: next((k for k, v in group_dict.items() if x in v), 0))
    new_col = new_col.rename(new_col.name + '_flag')
    new_cols = pd.get_dummies(new_col) # get dummies from group flag


   # create interaction dummies (group-dummy * rebatelag_dummy) iterating over each group_id and concatenating in the end:
    df_group_interactions = pd.DataFrame(df_column) # for initialization with correct index
    group_lst = group_dict.keys()
    for group_id in group_lst:
        dummies_group = reb_dummies_df.multiply(new_cols[group_id], axis = 'index')
        
        new_dummy_names = []
        for name in dummies_group.columns:
            new_dummy_names.append(name + '-j' + str(group_id))
        dummies_group.columns = new_dummy_names
        df_group_interactions = df_group_interactions.merge(dummies_group,left_index= True, right_index=True)
        #df_group_dummies.replace(np.NaN, 0, inplace = True)
    return df_group_interactions


get_grouping(houshold['TFTOTINC'], totinc_dict, houshold[reb_dummies_lst])

TFTOTINC  erebate-1-j1  erebate0-j1  erebate1-j1  \
SSUID        month_total                                                     
19128000276  6.0              1099           0.0          0.0          0.0   
             7.0              1099           0.0          0.0          0.0   
             8.0              1099           0.0          0.0          0.0   
             9.0              1099           0.0          0.0          0.0   
             10.0              381           0.0          0.0          0.0   
...                            ...           ...          ...          ...   
955958398688 10.0            10104           0.0          0.0          0.0   
             11.0            10333           0.0          0.0          0.0   
             12.0            10333           0.0          0.0          0.0   
             13.0            10333           0.0          0.0          0.0   
             14.0            10333           0.0          0.0          0.0   

                          erebate2-j1  erebate3-j1  erebate4-j1  erebate-1-j2  \
SSUID        month_total                                                        
19128000276  6.0                  0.0          0.0          0.0           0.0   
             7.0                  0.0          0.0          0.0           0.0   
             8.0                  0.0          0.0          0.0           0.0   
             9.0                  0.0          0.0          0.0           0.0   
             10.0                 0.0          0.0          0.0           0.0   
...                               ...          ...          ...           ...   
955958398688 10.0                 0.0          0.0          0.0           0.0   
             11.0                 0.0          0.0          0.0           0.0   
             12.0                 0.0          0.0          0.0           0.0   
             13.0                 0.0          0.0          0.0           0.0   
             14.0                 0.0          0.0          0.0           0.0   

                          erebate0-j2  erebate1-j2  ...  erebate1-j3  \
SSUID        month_total                            ...                
19128000276  6.0                  0.0          0.0  ...          0.0   
             7.0                  0.0          0.0  ...          0.0   
             8.0                  0.0          0.0  ...          0.0   
             9.0                  0.0          0.0  ...          0.0   
             10.0                 0.0          0.0  ...          0.0   
...                               ...          ...  ...          ...   
955958398688 10.0                 0.0          0.0  ...          1.0   
             11.0                 0.0          0.0  ...          0.0   
             12.0                 0.0          0.0  ...          0.0   
             13.0                 0.0          0.0  ...          0.0   
             14.0                 0.0          0.0  ...          0.0   

                          erebate2-j3  erebate3-j3  erebate4-j3  erebate-1-j4  \
SSUID        month_total                                                        
19128000276  6.0                  0.0          0.0          0.0           0.0   
             7.0                  0.0          0.0          0.0           0.0   
             8.0                  0.0          0.0          0.0           0.0   
             9.0                  0.0          0.0          0.0           0.0   
             10.0                 0.0          0.0          0.0           0.0   
...                               ...          ...          ...           ...   
955958398688 10.0                 0.0          0.0          0.0           0.0   
             11.0                 0.0          1.0          0.0           0.0   
             12.0                 0.0          0.0          1.0           0.0   
             13.0                 0.0          0.0          0.0           0.0   
             14.0                 0.0          0

In [34]:
test_group_dummies = get_grouping(houshold['TFTOTINC'], totinc_dict, houshold[reb_dummies_lst])

print(test_group_dummies.shape)
print(test_group_dummies['erebate3-j3'].sum())
reb_interaction_dummies = test_group_dummies.columns
reb_interaction_dummies

(209904, 25)
5534.0


Index(['TFTOTINC', 'erebate-1-j1', 'erebate0-j1', 'erebate1-j1', 'erebate2-j1',
       'erebate3-j1', 'erebate4-j1', 'erebate-1-j2', 'erebate0-j2',
       'erebate1-j2', 'erebate2-j2', 'erebate3-j2', 'erebate4-j2',
       'erebate-1-j3', 'erebate0-j3', 'erebate1-j3', 'erebate2-j3',
       'erebate3-j3', 'erebate4-j3', 'erebate-1-j4', 'erebate0-j4',
       'erebate1-j4', 'erebate2-j4', 'erebate3-j4', 'erebate4-j4'],
      dtype='object')

In [ ]:
df_w_group_interaction = test_group_dummies.merge(houshold[['TFEARN', 'ERBATAMT']], left_index=True, right_index=True)
df_w_group_interaction['month_total'] = pd.Categorical(houshold['month_total'])
df_w_group_interaction

In [ ]:
# with interaction dummies for total income groups:
exog_vars = df_w_group_interaction[['erebate-1-j1', 'erebate0-j1', 'erebate1-j1', 'erebate2-j1',
                                    'erebate3-j1', 'erebate4-j1', 'erebate-1-j2', 'erebate0-j2',
                                    'erebate1-j2', 'erebate2-j2', 'erebate3-j2', 'erebate4-j2',
                                   'erebate-1-j3', 'erebate0-j3', 'erebate1-j3', 'erebate2-j3',
                                   'erebate3-j3', 'erebate4-j3', 'erebate-1-j4', 'erebate0-j4',
                                   'erebate1-j4', 'erebate2-j4', 'erebate3-j4', 'erebate4-j4']]#.multiply(df_w_group_interaction['ERBATAMT'], axis = 'index')
#exog_vars['month_total'] = pd.Categorical(houshold['month_total'])

exog = sm.add_constant(exog_vars)

model = PanelOLS(houshold['TFEARN'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()

# 2SLS approach by hand

In [209]:
# analysis on subset with interactions terms:
powell_hh = houshold[['ERBATAMT', 'TFEARN']]
powell_hh['reb'] = houshold['erebate-1'] + houshold['erebate0']
powell_hh['reb_lag'] = houshold['erebate1'] + houshold['erebate2']
powell_hh['erbatamt'] = powell_hh['ERBATAMT'].multiply(powell_hh['reb'], axis = 'index')  # so that it is 0 in the months without rebate payout
powell_hh['erbatamt_lag'] = powell_hh['ERBATAMT'].multiply(powell_hh['reb_lag'], axis = 'index')
#interaction terms based on  marital status, household size, month (and month relative to interview month)
powell_hh[['ems', 'EHHNUMPP']] = houshold[['ems', 'EHHNUMPP']]

ems_interactions = powell_hh[['reb', 'reb_lag']].multiply(powell_hh['ems'], axis='index')
ems_interactions.columns = ['ems_reb', 'ems_reb_lag']

ehhnumpp_dict = {1:range(1,2), 2:range(2,3), 3: range(3,4), 4: range(5,20)} # group 4 has to be omitted to get full rank
hhsize_interactions = get_grouping(powell_hh['EHHNUMPP'], ehhnumpp_dict, powell_hh[['reb', 'reb_lag']])
powell_hh = pd.concat([powell_hh,ems_interactions,hhsize_interactions], axis = 1)
powell_hh.columns

<ipython-input-209-02b3efd02e10>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb'] = houshold['erebate-1'] + houshold['erebate0']
<ipython-input-209-02b3efd02e10>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb_lag'] = houshold['erebate1'] + houshold['erebate2']
<ipython-input-209-02b3efd02e10>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

Index(['ERBATAMT', 'TFEARN', 'reb', 'reb_lag', 'erbatamt', 'erbatamt_lag',
       'ems', 'EHHNUMPP', 'ems_reb', 'ems_reb_lag', 'EHHNUMPP', 'reb-j1',
       'reb_lag-j1', 'reb-j2', 'reb_lag-j2', 'reb-j3', 'reb_lag-j3', 'reb-j4',
       'reb_lag-j4'],
      dtype='object')

### strategy

- get fitted values for reb_amount and reb_amount_lag
- plug into second stage

In [210]:
# rebate amount with reb and reb lag only 
exog_vars = powell_hh[['reb', 'reb_lag']]
exog = sm.add_constant(exog_vars)

model_base_reb = PanelOLS(powell_hh['erbatamt'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_base_reb = model_base_reb.fit()
params_base_reb.params

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


const        0.299183
reb        968.479283
reb_lag     -0.014709
Name: parameter, dtype: float64

In [211]:
# rebate amount with reb and reb lag only 
exog_vars = powell_hh[['reb', 'reb_lag']]
exog = sm.add_constant(exog_vars)

model_base_reb_lag = PanelOLS(powell_hh['erbatamt_lag'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_base_reb_lag = model_base_reb_lag.fit()
params_base_reb_lag.params

const        1.133578
reb         -6.986479
reb_lag    975.365140
Name: parameter, dtype: float64

In [194]:
# first stage fitted rebate with interactions 
exog_vars = powell_hh[['reb', 'reb_lag', 'reb-j1', 'reb_lag-j1', 'reb-j2', 'reb_lag-j2', 'reb-j3', 'reb_lag-j3', 'ems_reb', 'ems_reb_lag']]
exog = sm.add_constant(exog_vars)
exog
model_interactions = PanelOLS(powell_hh['ERBATAMT'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_interactions = model_interactions.fit()
params_interactions

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,ERBATAMT,R-squared:,0.8147
Estimator:,PanelOLS,R-squared (Between):,0.8041
No. Observations:,209904,R-squared (Within):,0.8526
Date:,"Fri, Jun 03 2022",R-squared (Overall):,0.8445
Time:,11:03:27,Log-likelihood,-1.263e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,8.077e+04
Entities:,26238,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(10,183646)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,8.077e+04


In [212]:
# generate fitted values for the second stage regression:
print(params_base_reb.params)
erbatamt_fitted_base_reb = params_base_reb.fitted_values
erbatamt_fitted_base_reb.columns = ['erbatamt_fitted_base_reb']

print(params_base_reb_lag.params)
erbatamt_fitted_base_reb_lag = params_base_reb_lag.fitted_values
erbatamt_fitted_base_reb_lag.columns = ['erbatamt_fitted_base_reb_lag']

powell_hh = pd.concat([powell_hh,erbatamt_fitted_base_reb, erbatamt_fitted_base_reb_lag], axis = 1)
powell_hh.head()

const        0.299183
reb        968.479283
reb_lag     -0.014709
Name: parameter, dtype: float64
const        1.133578
reb         -6.986479
reb_lag    975.365140
Name: parameter, dtype: float64


ERBATAMT  TFEARN  reb  reb_lag  erbatamt  \
SSUID       month_total                                             
19128000276 6.0               0.0       0  0.0      0.0       0.0   
            7.0               0.0       0  0.0      0.0       0.0   
            8.0               0.0       0  0.0      0.0       0.0   
            9.0               0.0       0  0.0      0.0       0.0   
            10.0              0.0       0  0.0      0.0       0.0   

                         erbatamt_lag  ems  EHHNUMPP  ems_reb  ems_reb_lag  \
SSUID       month_total                                                      
19128000276 6.0                   0.0    1         3      0.0          0.0   
            7.0                   0.0    1         3      0.0          0.0   
            8.0                   0.0    1         3      0.0          0.0   
            9.0                   0.0    1         3      0.0          0.0   
            10.0                  0.0    1         3      0.0          0.0   

                         ...  reb-j1  reb_lag-j1  reb-j2  reb_lag-j2  reb-j3  \
SSUID       month_total  ...                                                   
19128000276 6.0          ...     0.0         0.0     0.0         0.0     0.0   
            7.0          ...     0.0         0.0     0.0         0.0     0.0   
            8.0          ...     0.0         0.0     0.0         0.0     0.0   
            9.0          ...     0.0         0.0     0.0         0.0     0.0   
            10.0         ...     0.0         0.0     0.0         0.0     0.0   

                         reb_lag-j3  reb-j4  reb_lag-j4  \
SSUID       month_total                                   
19128000276 6.0                 0.0     0.0         0.0   
            7.0                 0.0     0.0         0.0   
            8.0                 0.0     0.0         0.0   
            9.0                 0.0     0.0         0.0   
            10.0                0.0     0.0         0.0   

                         erbatamt_fitted_base_reb  \
SSUID       month_total                             
19128000276 6.0                          0.299183   
            7.0                          0.299183   
            8.0                          0.299183   
            9.0                          0.299183   
            10.0                         0.299183   

                         erbatamt_fitted_base_reb_lag  
SSUID       month_total                                
19128000276 6.0                              1.133578  
            7.0                              1.133578  
            8.0                              1.133578  
            9.0                              1.133578  
            10.0                             1.133578  

[5 rows x 21 columns]

## next steps:

- second stage regression: est_reb, est_reb_lag by the .predict() method

In [213]:
exog_vars = powell_hh[['erbatamt_fitted_base_reb', 'erbatamt_fitted_base_reb_lag']]
exog = sm.add_constant(exog_vars)
exog
model_interactions = PanelOLS(powell_hh['TFEARN'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_interactions = model_interactions.fit()
params_interactions

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,TFEARN,R-squared:,3.197e-05
Estimator:,PanelOLS,R-squared (Between):,-0.0001
No. Observations:,209904,R-squared (Within):,-6.502e-05
Date:,"Fri, Jun 03 2022",R-squared (Overall):,-0.0001
Time:,11:34:48,Log-likelihood,-1.895e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.9360
Entities:,26238,P-value,0.0531
Avg Obs:,8.0000,Distribution:,"F(2,183654)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,2.9360
